In [22]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import json
import re
import nltk
import json
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
import emoji
import unicodedata

In [23]:
df = pd.read_csv('data_250.csv')
df

,full_text,case_folding CPMK2,preprocessing CPMK3,label
0,yeay naik secret elshark gran maja 1/4m pulau ...,yeay naik secret elshark gran maja 1/4m pulau ...,yeay secret elshark gran maja pulau pulang mar...,positive
1,semenjak kenal fish aku melupakan gunung,semenjak kenal fish aku melupakan gunung,semenjak kenal lupa gunung,negative
2,baru banget aku main fish semalam langsung keb...,baru banget aku main fish semalam langsung keb...,main malam langsung bawa mimpi,negative
3,mending mancing fish,mending mancing fish,pancing,positive
4,dunia ini sudah gila kayanya ditiktok gue nyar...,dunia ini sudah gila kayanya ditiktok gue nyar...,dunia gila kaya ditiktok cari kraken mythology...,negative
...,...,...,...,...
245,pengen nyobain ikutan fish tapi takut akun aku...,pengen nyobain ikutan fish tapi takut akun aku...,ken nyobain ikut takut akun bau level,neutral
246,butuh banget teman main fish bingung kalau dap...,butuh banget teman main fish bingung kalau dap...,butuh teman main bingung secret pamer,neutral
247,dari fish kita belajar semua punya proses masi...,dari fish kita belajar semua punya proses masi...,ajar proses left right arrow,neutral
248,kangen banget fish yang update tiap jam subuh ...,kangen banget fish yang update tiap jam subuh ...,kangen update jam subuh huwaa,neutral


<h>PREPOCESSING</h>

In [24]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
stop_words = set(stopwords.words("indonesian"))

stopwords_extra = {
    'aku', 'nya', 'oke', 'aja', 'saya', 'sih', 'doang', 'gw', 'gua', 'aq', 
    'kamu', 'lu', 'loe', 'km', 'kmu', 'iya', 'ya', 'yah', 'nih', 'dong', 
    'lah', 'kah', 'deh', 'ajah', 'aj', 'gak', 'ga', 'gk', 'ngga', 'tp', 
    'tpi', 'sm', 'sma', 'lg', 'lgi', 'udh', 'udah', 'dah', 'blm', 'blom',
    'klo', 'kalo', 'kl', 'gt', 'gtu', 'gini', 'gmn', 'gimana', 'wkwk', 
    'haha', 'hehe', 'huhu', 'anjir', 'anjing', 'bjir', 'cok', 'coy', 
    'dm', 'btw', 'pls', 'plis', 'brp', 'brpa', 'kak', 'bang', 'bro', 
    'sis', 'pun', 'mah', 'tu', 'nih', 'https', 'http', 'www', 'co', 
    'id', 'net', 'org', 'yuk', 'ken', "gue", "guys", "tau", "tuh", "daki",
    "biar", "kali", "banget", 'mending', 'gitu', "kakak", "pas", "karena",
    "ayo", "semoga", "kena", "ajaa", "gara gara", "sender", "gara", "pake", 
    "emang", "txtfromroblox", "by the way", "karna", "avqris avrekber okii"
}

stop_words.update(stopwords_extra)

import json

In [25]:
with open("koreksi.json", "r", encoding="utf-8") as f:
    abbreviation_dict = json.load(f)

bentuk_kata = re.compile(r'^[a-z]+$')

def reduce_repeated_chars(word):
    return re.sub(r'(.)\1{2,}', r'\1', word)

def remove_non_latin(text):
    return ''.join(
        c for c in text 
        if unicodedata.category(c)[0] != 'C'
        and "LATIN" in unicodedata.name(c, 'LATIN')
    )

def preprocess(text):
    if pd.isna(text):
        return ""

def remove_non_latin(text):
    return re.sub(r'[^\x00-\x7F]+', ' ', text)  

def preprocess(text):
    if pd.isna(text):
        return ""

    text = text.lower()
    text = re.sub(r'http\S+|www\S+', ' ', text)
    text = emoji.replace_emoji(text, replace=' ')
    text = re.sub(r'[@#]\w+', ' ', text)

    text = remove_non_latin(text)

    text = re.sub(r'[^a-z\s]', ' ', text)

    words = text.split()
    cleaned = []

    for w in words:
        w = reduce_repeated_chars(w)

        if w in abbreviation_dict:
            w = abbreviation_dict[w]

        if not bentuk_kata.match(w):
            continue

        if w in stop_words:
            continue

        w = stemmer.stem(w)

        if w in stop_words:
            continue

        cleaned.append(w)

    cleaned_text = " ".join(cleaned)
    return cleaned_text


In [26]:
df["bersih"] = df["preprocessing CPMK3"].apply(preprocess)
df['bersih']

0      yeay secret elshark gran maja pulau pulang mar...
1                             semenjak kenal lupa gunung
2                         main malam langsung bawa mimpi
3                                                pancing
4      dunia gila kaya ditiktok cari kraken mythology...
                             ...                        
245                         nyobain takut akun bau level
246                butuh teman main bingung secret pamer
247                         ajar proses left right arrow
248                        kangen update jam subuh huwaa
249    urus map untung tinggal fitur fuck racun fishi...
Name: bersih, Length: 250, dtype: object

<h>EKSPERIMEN KLASIFIKASI</h>

<h>SVM</h>

1. SVM + TF-IDF

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

# ============================
# 1. LOAD DATASET
# ============================

# Pisahkan fitur dan label
X = df['bersih']
y = df['label']

# ============================
# 2. SPLIT DATA TRAIN & TEST
# ============================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ============================
# 3. TF-IDF FEATURE EXTRACTION
# ============================
tfidf = TfidfVectorizer(
    max_features=5000,      
    ngram_range=(1,2),      
    stop_words=None         
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf  = tfidf.transform(X_test)

# ============================
# 4. TRAIN MODEL SVM
# ============================
model = LinearSVC()
model.fit(X_train_tfidf, y_train)

# ============================
# 5. PREDIKSI
# ============================
y_pred = model.predict(X_test_tfidf)

# ============================
# 6. EVALUASI
# ============================
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Akurasi:", accuracy)
print("\nClassification Report:")
print(report)


Akurasi: 0.54

Classification Report:
              precision    recall  f1-score   support

    negative       0.57      0.63      0.60        19
     neutral       0.20      0.25      0.22         8
    positive       0.68      0.57      0.62        23

    accuracy                           0.54        50
   macro avg       0.49      0.48      0.48        50
weighted avg       0.56      0.54      0.55        50



2. SVM + BOW

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

# ============================
# 1. LOAD DATASET
# ============================

X = df['bersih']     # fitur teks
y = df['label']      # label sentimen

# ============================
# 2. SPLIT DATA
# ============================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ============================
# 3. EKSTRAKSI FITUR: BAG OF WORDS
# ============================
bow = CountVectorizer(
    max_features=5000,     # batasi jumlah fitur
    ngram_range=(1,2),     # unigram + bigram
    stop_words=None
)

X_train_bow = bow.fit_transform(X_train)
X_test_bow  = bow.transform(X_test)

# ============================
# 4. MODEL SVM
# ============================
model = LinearSVC()
model.fit(X_train_bow, y_train)

# ============================
# 5. PREDIKSI
# ============================
y_pred = model.predict(X_test_bow)

# ============================
# 6. EVALUASI
# ============================
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("=======================================")
print("Akurasi BoW + SVM:", accuracy)
print("=======================================")
print("\nClassification Report:\n")
print(report)


Akurasi BoW + SVM: 0.62

Classification Report:

              precision    recall  f1-score   support

    negative       0.62      0.65      0.63        20
     neutral       1.00      0.10      0.18        10
    positive       0.61      0.85      0.71        20

    accuracy                           0.62        50
   macro avg       0.74      0.53      0.51        50
weighted avg       0.69      0.62      0.57        50



<h>LOGISTIC REGRESION</h>

In [30]:
pip install gensim

   ---------------------------------------- 0.0/24.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.4 MB 660.6 kB/s eta 0:00:37
   ---------------------------------------- 0.0/24.4 MB 495.5 kB/s eta 0:00:50
   ---------------------------------------- 0.1/24.4 MB 491.5 kB/s eta 0:00:50
   ---------------------------------------- 0.1/24.4 MB 722.1 kB/s eta 0:00:34
   ---------------------------------------- 0.2/24.4 MB 1.0 MB/s eta 0:00:24
   ---------------------------------------- 0.2/24.4 MB 1.0 MB/s eta 0:00:24
   ---------------------------------------- 0.2/24.4 MB 1.0 MB/s eta 0:00:24
    --------------------------------------- 0.3/24.4 MB 863.3 kB/s eta 0:00:28
    --------------------------------------- 0.5/24.4 MB 1.3 MB/s eta 0:00:18
   - -------------------------------------- 0.6/24.4 MB 1.4 MB/s eta 0:00:17
   - -------------------------------------- 0.7/24.4 MB 1.3 MB/s eta 0:00:19
   - -------------------------------------- 0.7/24.4 MB 1.3 MB/s eta 0:00


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


1. LOGISTIC REGRESION + WORD2VEC

In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from gensim.models import Word2Vec
import numpy as np

# ============================
# 1. LOAD DATASET
# ============================
X = df['bersih']
y = df['label']

# Tokenisasi sederhana (Word2Vec butuh token list)
tokenized = X.apply(lambda x: x.split())

# ============================
# 2. SPLIT DATA
# ============================
X_train_tokens, X_test_tokens, y_train, y_test = train_test_split(
    tokenized, y, test_size=0.2, random_state=42, stratify=y
)

# ============================
# 3. TRAIN WORD2VEC
# ============================
w2v_model = Word2Vec(
    sentences=X_train_tokens,
    vector_size=100,   # dimensi vektor
    window=5,
    min_count=2,
    workers=4
)

# ============================
# 4. FUNGSI RATA-RATA VEKTOR WORD2VEC
# ============================
def document_vector(tokens):
    """Rata-rata vektor Word2Vec dari semua kata dalam 1 dokumen."""
    vectors = [
        w2v_model.wv[word] 
        for word in tokens 
        if word in w2v_model.wv
    ]
    if len(vectors) == 0:
        return np.zeros(w2v_model.vector_size)   # jika tidak ada kata yang dikenal
    return np.mean(vectors, axis=0)

# Convert semua dokumen ke vektor numeric
X_train_vec = np.array([document_vector(tokens) for tokens in X_train_tokens])
X_test_vec  = np.array([document_vector(tokens) for tokens in X_test_tokens])

# ============================
# 5. MODEL LOGISTIC REGRESSION
# ============================
logreg = LogisticRegression(
    max_iter=2000,
    solver='lbfgs',
    multi_class='auto'
)

logreg.fit(X_train_vec, y_train)

# ============================
# 6. PREDIKSI
# ============================
y_pred = logreg.predict(X_test_vec)

# ============================
# 7. EVALUASI
# ============================
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("=======================================")
print("Akurasi Word2Vec + Logistic Regression:", accuracy)
print("=======================================")
print("\nClassification Report:\n")
print(report)


Akurasi Word2Vec + Logistic Regression: 0.46

Classification Report:

              precision    recall  f1-score   support

    negative       0.44      0.55      0.49        20
     neutral       0.00      0.00      0.00        10
    positive       0.48      0.60      0.53        20

    accuracy                           0.46        50
   macro avg       0.31      0.38      0.34        50
weighted avg       0.37      0.46      0.41        50



2. LOGISTIC REGRESION + TF-IDF

In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# ============================
# 1. LOAD DATASET
# ============================
X = df['bersih']
y = df['label']

# ============================
# 2. SPLIT DATA
# ============================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ============================
# 3. TF-IDF FEATURE EXTRACTION
# ============================
tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1,2),     # unigram + bigram
    sublinear_tf=True,     # tf smoothing
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf  = tfidf.transform(X_test)

# ============================
# 4. MODEL LOGISTIC REGRESSION
# ============================
logreg = LogisticRegression(
    max_iter=2000,
    solver='lbfgs',
    multi_class='auto'
)

logreg.fit(X_train_tfidf, y_train)

# ============================
# 5. PREDIKSI
# ============================
y_pred = logreg.predict(X_test_tfidf)

# ============================
# 6. EVALUASI
# ============================
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("=======================================")
print("Akurasi TF-IDF + Logistic Regression:", accuracy)
print("=======================================")
print("\nClassification Report:\n")
print(report)


Akurasi TF-IDF + Logistic Regression: 0.64

Classification Report:

              precision    recall  f1-score   support

    negative       0.59      0.80      0.68        20
     neutral       0.00      0.00      0.00        10
    positive       0.70      0.80      0.74        20

    accuracy                           0.64        50
   macro avg       0.43      0.53      0.48        50
weighted avg       0.52      0.64      0.57        50



<h>NAIVE BAYES</h>

1. NAIVE BAYES + BOW

In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# ============================
# 1. LOAD DATASET
# ============================
X = df['bersih']
y = df['label']

# ============================
# 2. SPLIT DATA
# ============================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ============================
# 3. BAG OF WORDS (BoW)
# ============================
bow = CountVectorizer(
    max_features=5000,
    ngram_range=(1,2),   # unigram + bigram
    stop_words=None
)

X_train_bow = bow.fit_transform(X_train)
X_test_bow  = bow.transform(X_test)

# ============================
# 4. MODEL NAIVE BAYES
# ============================
nb_model = MultinomialNB()
nb_model.fit(X_train_bow, y_train)

# ============================
# 5. PREDIKSI
# ============================
y_pred = nb_model.predict(X_test_bow)

# ============================
# 6. EVALUASI
# ============================
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("=======================================")
print("Akurasi BoW + Naive Bayes:", accuracy)
print("=======================================")
print("\nClassification Report:\n")
print(report)


Akurasi BoW + Naive Bayes: 0.58

Classification Report:

              precision    recall  f1-score   support

    negative       0.56      0.75      0.64        20
     neutral       0.33      0.10      0.15        10
    positive       0.65      0.65      0.65        20

    accuracy                           0.58        50
   macro avg       0.51      0.50      0.48        50
weighted avg       0.55      0.58      0.55        50



2. NAIVE BAYES + WOR2VEC

In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report
from gensim.models import Word2Vec

# ============================
# 1. LOAD DATASET
# ============================
X = df['bersih']
y = df['label']

# Tokenisasi sederhana (untuk Word2Vec)
sentences = [text.split() for text in X]

# ============================
# 2. TRAIN WORD2VEC MODEL
# ============================
w2v_model = Word2Vec(
    sentences,
    vector_size=100,
    window=5,
    min_count=2,
    workers=4
)

# ============================
# 3. FUNGSI RATA-RATA VEKTOR
# ============================
def vectorize_sentence(sentence):
    words = sentence.split()
    vectors = [
        w2v_model.wv[word] for word in words if word in w2v_model.wv
    ]
    if len(vectors) == 0:
        return np.zeros(100)   # Jika kalimat tidak punya kata dalam vocab
    return np.mean(vectors, axis=0)

# Konversi seluruh dataset ke vektor
X_vectors = np.array([vectorize_sentence(text) for text in X])

# ============================
# 4. SPLIT DATA
# ============================
X_train, X_test, y_train, y_test = train_test_split(
    X_vectors, y, test_size=0.2, random_state=42, stratify=y
)

# ============================
# 5. MODEL NAIVE BAYES (GaussianNB)
# ============================
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

# ============================
# 6. PREDIKSI & EVALUASI
# ============================
y_pred = nb_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("=======================================")
print("Akurasi Word2Vec + Naive Bayes:", accuracy)
print("=======================================")
print("\nClassification Report:\n")
print(report)


Akurasi Word2Vec + Naive Bayes: 0.42

Classification Report:

              precision    recall  f1-score   support

    negative       0.40      0.30      0.34        20
     neutral       0.20      0.30      0.24        10
    positive       0.60      0.60      0.60        20

    accuracy                           0.42        50
   macro avg       0.40      0.40      0.39        50
weighted avg       0.44      0.42      0.43        50



<h>RANDOM FOREST</h>

1. RANDOM FOREST + TF-IDF

In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# ============================
# 1. LOAD DATASET
# ============================
X = df['bersih']
y = df['label']

# ============================
# 2. SPLIT DATA
# ============================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ============================
# 3. TF-IDF FEATURE EXTRACTION
# ============================
tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),   # unigram + bigram
    stop_words=None
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf  = tfidf.transform(X_test)

# ============================
# 4. RANDOM FOREST MODEL
# ============================

rf_model = RandomForestClassifier(
    n_estimators=300,        # jumlah pohon
    max_depth=None,         # bebas, biarkan model cari sendiri
    random_state=42,
    n_jobs=-1               # pakai semua CPU
)

rf_model.fit(X_train_tfidf, y_train)

# ============================
# 5. PREDIKSI
# ============================
y_pred = rf_model.predict(X_test_tfidf)

# ============================
# 6. EVALUASI
# ============================
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("=======================================")
print("Akurasi TF-IDF + Random Forest:", accuracy)
print("=======================================")
print("\nClassification Report:\n")
print(report)


Akurasi TF-IDF + Random Forest: 0.56

Classification Report:

              precision    recall  f1-score   support

    negative       0.62      0.50      0.56        20
     neutral       0.00      0.00      0.00        10
    positive       0.55      0.90      0.68        20

    accuracy                           0.56        50
   macro avg       0.39      0.47      0.41        50
weighted avg       0.47      0.56      0.49        50



2. RANDOM FOREST + BOW

In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# ============================
# 1. LOAD DATASET
# ============================
X = df['bersih']
y = df['label']

# ============================
# 2. TRAIN TEST SPLIT
# ============================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ============================
# 3. FEATURE EXTRACTION: BOW
# ============================
bow_vectorizer = CountVectorizer(
    max_features=5000,
    ngram_range=(1, 2),    # unigram + bigram
    stop_words=None
)

X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow  = bow_vectorizer.transform(X_test)

# ============================
# 4. RANDOM FOREST MODEL
# ============================
rf_model = RandomForestClassifier(
    n_estimators=300,   
    max_depth=None,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train_bow, y_train)

# ============================
# 5. PREDIKSI
# ============================
y_pred = rf_model.predict(X_test_bow)

# ============================
# 6. EVALUASI
# ============================
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("=======================================")
print("Akurasi BoW + Random Forest:", accuracy)
print("=======================================")
print("\nClassification Report:\n")
print(report)


Akurasi BoW + Random Forest: 0.5

Classification Report:

              precision    recall  f1-score   support

    negative       0.56      0.25      0.34        20
     neutral       0.00      0.00      0.00        10
    positive       0.49      1.00      0.66        20

    accuracy                           0.50        50
   macro avg       0.35      0.42      0.33        50
weighted avg       0.42      0.50      0.40        50

